# Inspection methods

> This module defines all available inspection methods (= inspection options)

Note: This submodule does not use the abstract base classes `ProcessingObject` and `ProcessingStrategy`. The reason for this is to make specifying the settings and configurations for these inspection methods more interactive for the user - especially when accessed via the GUI. Unfortunately, this sacrifices a bit of overall code consistency. However, since inspection also does not really represent a processing step, this trade-off in favor of usability seemed reasonable. To make make the distinction ever more apparent, the classes that handle the different inspections will be referred to as "methods" instead of "strategies" in the class names.

In [ ]:
#| default_exp inspection/methods

In [ ]:
#| export

from typing import Tuple, List, Dict
from pathlib import Path

import numpy as np
from skimage import io, color

from findmycells.database import Database
from findmycells import utils

from matplotlib.backend_bases import MouseButton
import matplotlib.pyplot as plt

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

class InspectionMethod:
    
    
    def __init__(self, file_id: str, area_roi_id: str, database: Database) -> None:
        self.file_id = file_id
        self.area_roi_id = area_roi_id
        self.database = database
        self.preprocessed_image = self._load_preprocessed_image()
        self.postprocessed_segmentation_mask = self._load_postprocessed_segmentation_mask()
        self.rgb_color_coded_2d_overlay_of_image_and_mask = self._create_rgb_color_coded_2d_overlay_of_image_and_mask()
        self.area_roi = self.database.area_rois_for_quantification[file_id]['all_planes'][area_roi_id]

    
    def _load_preprocessed_image(self) -> np.ndarray:
        preprocessed_images_dir_path = self.database.project_configs.root_dir.joinpath(self.database.preprocessed_images_dir)
        return utils.load_zstack_as_array_from_single_planes(path = preprocessed_images_dir_path, file_id = self.file_id)


    def _load_postprocessed_segmentation_mask(self) -> np.ndarray:
        postprocessed_masks_dir_path = self.database.project_configs.root_dir.joinpath(self.database.quantified_segmentations_dir, self.area_roi_id)
        return utils.load_zstack_as_array_from_single_planes(path = postprocessed_masks_dir_path, file_id = self.file_id)
    
    
    def get_available_label_ids(self) -> List[int]:
        return list(np.unique(self.postprocessed_segmentation_mask))
    
    
    def get_available_multi_match_idxs(self) -> List[int]:
        return self.database.multi_matches_traceback[self.file_id]['final_label_id']
    
    
    def get_center_coords_from_mouse_click_position(self) -> Tuple[int, int]:
        from IPython import get_ipython
        ipy = get_ipython()
        if ipy is not None:
            ipy.run_line_magic('matplotlib', 'tk')
        fig = plt.figure(figsize=(10, 10), facecolor = 'white')
        plt.connect('button_press_event', self._matplotlib_figure_clicked)
        plt.imshow(self.rgb_color_coded_2d_overlay_of_image_and_mask)

        
    def _matplotlib_figure_clicked(self, event):
        if event.button is MouseButton.RIGHT:
            # only alternative to printing the values (without installing additional packages) seems to be a global variable...
            print(f'Your selected x-coordinate was: {event.x}, and your selected y-coordinate was: {event.y}.')
            plt.close()
    
    
    def get_center_coords_from_label_id(self, label_id: int) -> Tuple[int, int]:
        if self.postprocessed_segmentation_mask.shape[0] > 1:
            mask_as_single_plane = np.max(self.postprocessed_segmentation_mask, axis=0)
        else:
            mask_as_single_plane = self.postprocessed_segmentation_mask
        feature_roi = utils.get_polygon_from_instance_segmentation(single_plane = mask_as_single_plane, label_id = label_id)
        return (feature_roi.centroid.y, feature_roi.centroid.x)


    def get_center_coords_from_multi_match_idx(self, multi_match_idx: int) -> Tuple[int, int]:
        label_id = self.database.multi_matches_traceback[self.file_id]['final_label_id'][multi_match_idx]
        return self.get_center_coords_from_label_id(label_id = label_id)
    
    
    def _create_rgb_color_coded_2d_overlay_of_image_and_mask(self) -> np.ndarray:
        max_projection_image = np.max(self.preprocessed_image, axis=0)
        max_projection_mask = np.max(self.postprocessed_segmentation_mask, axis=0)
        return color.label2rgb(max_projection_mask, image = max_projection_image, bg_label = 0, bg_color = None, saturation = 1, alpha = 1)    
    


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()